In [1]:
import os
#import random
#import pandas as pd
import numpy as np
#import glob
#import matplotlib.pyplot as plt
#plt.style.use("ggplot")
#%matplotlib inline

from sklearn.model_selection import KFold, train_test_split
import cv2

from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.optimizers import Adam

from keras_unet_collection import models #Keras unet collection
#import tensorflow as tf
from PIL import Image
from datetime import datetime
from keras.utils import to_categorical


## DL R2-Unet without k-fold

In [2]:
#define directory where images and masks are located on local disk
image_directory = 'D:/UniBas/Bachelorarbeit/some_images/'
mask_directory = 'D:/UniBas/Bachelorarbeit/some_masks/'

#define the properties and empty list for resized images and masks
SIZE = 256
image_dataset = []
mask_dataset = []


#define custom functions
def IoU(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true,-1) + K.sum(y_pred,-1) - intersection
    iou = (intersection + smooth) / ( union + smooth)
    return iou

#enumerate and resize images/masks
images = os.listdir(image_directory)
for i, image_name in enumerate(images):    #enumerate method adds a counter and returns the enumerate object
    if (image_name.split('.')[1] == 'tif'):
        #print(image_directory+image_name)
        image = cv2.imread(image_directory+image_name, 1)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        image_dataset.append(np.array(image))


masks = os.listdir(mask_directory)
for i, image_name in enumerate(masks):
    if (image_name.split('.')[1] == 'tif'):
        image = cv2.imread(mask_directory+image_name, 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        mask_dataset.append(np.array(image))

#define some hyperparameters
num_labels = 2  #Binary classificaion (missmatch with literature/code examples!)
batch_size = 2  #keep it smaller than 3
epochs = 2

#normalize images
image_dataset = np.array(image_dataset)/255
#do not normalize masks, just rescale to 0 to 1.
mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255
mask_dataset = to_categorical(mask_dataset)

#define callback function 
callbacks = [
  EarlyStopping(patience=8, verbose=1),
  ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.00001, verbose=1),
  ModelCheckpoint('A_modelr2V2.h5', verbose=1, save_best_only=True, save_weights_only=False), # Give the model a name (the .h5 part)
  CSVLogger('A_modelr2V2.csv', separator=',', append=False)]                                  # Give the CSV-File a name! (X.csv)

#define the model architecture
#r2_unet_2d requires depth >= 2
model = models.r2_unet_2d((256, 256, 3), [64, 128, 256, 512], n_labels=num_labels,
                        stack_num_down=4, stack_num_up=4, recur_num=4,
                        activation='ReLU', output_activation='Softmax', 
                        batch_norm=True, pool='max', unpool='nearest', name='r2unet')

#compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(lr = 1e-3), 
            metrics=['accuracy', IoU])

#split dataset into training and validation set
X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size = 0.10, random_state = 0)

#start time meauserment
start2 = datetime.now() 

#fit model on data
resunet_history = model.fit(X_train, y_train, 
                    verbose=1,
                    batch_size = batch_size,
                    validation_data=(X_test, y_test), 
                    shuffle=False,
                    epochs=epochs,
                    callbacks=callbacks)

#stop time measurement and print the execution time
stop2 = datetime.now()
execution_time = stop2-start2
print("R2-Unet fitting time is: ", execution_time)

Epoch 1/2
5/5 [==============================] - 133s 23s/step - loss: 1.1136 - accuracy: 0.4963 - IoU: 0.6289 - val_loss: 14.0790 - val_accuracy: 0.0820 - val_IoU: 0.3880

Epoch 00001: val_loss improved from inf to 14.07903, saving model to A_modelr2V2.h5
Epoch 2/2
5/5 [==============================] - 112s 22s/step - loss: 0.2625 - accuracy: 0.9013 - IoU: 0.8846 - val_loss: 0.9594 - val_accuracy: 0.9374 - val_IoU: 0.9583

Epoch 00002: val_loss improved from 14.07903 to 0.95939, saving model to A_modelr2V2.h5
R2-Unet fitting time is:  0:04:33.256932


## DL R2-Unet with k-fold

In [3]:
#define directory where images and masks are located on local disk
image_directory = 'D:/UniBas/Bachelorarbeit/DeepACSA_images_VL/little_images/'
mask_directory = 'D:/UniBas/Bachelorarbeit/DeepACSA_masks_VL/little_masks/'

#define the properties and empty list for resized images and masks
SIZE = 256
image_dataset = []
mask_dataset = []


#define custom functions
def IoU(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true,-1) + K.sum(y_pred,-1) - intersection
    iou = (intersection + smooth) / ( union + smooth)
    return iou

#enumerate and resize images/masks
images = os.listdir(image_directory)
for i, image_name in enumerate(images):    #enumerate method adds a counter and returns the enumerate object
    if (image_name.split('.')[1] == 'tif'):
        #print(image_directory+image_name)
        image = cv2.imread(image_directory+image_name, 1)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        image_dataset.append(np.array(image))


masks = os.listdir(mask_directory)
for i, image_name in enumerate(masks):
    if (image_name.split('.')[1] == 'tif'):
        image = cv2.imread(mask_directory+image_name, 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        mask_dataset.append(np.array(image))

#define some hyperparameters
num_labels = 1  #Binary classificaion (missmatch on coding examples! 1 or 2 for binary classification)
batch_size = 2  #keep it smaller than 3
epochs = 60
num_folds = 5   #define the number of folds (usually 5-10 folds)

#normalize images
image_dataset = np.array(image_dataset)/255.
#do not normalize masks, just rescale to 0 to 1.
mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255.

#define K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=False)

#define per-fold score containers 
acc_per_fold = []
loss_per_fold = []
IoU_per_fold = []

#K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(image_dataset, mask_dataset):
  callbacks = [
    EarlyStopping(patience=8, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.00001, verbose=1),
    ModelCheckpoint(f'K-foldno{fold_no}-model_r2-VL-256.h5', verbose=1, save_best_only=True, save_weights_only=False), # Give the model a name (the .h5 part)
    CSVLogger(f'K-foldno{fold_no}-model_r2-VL-256.csv', separator=',', append=False)]                                  # Give the CSV file a name (.csv)

  #define the model architecture
  #r2_unet_2d requires depth >= 2
  model = models.r2_unet_2d((256, 256, 3), [64, 128, 256, 512], n_labels=num_labels,
                          stack_num_down=2, stack_num_up=1, recur_num=2,
                          activation='ReLU', output_activation='Softmax', 
                          batch_norm=True, pool='max', unpool='nearest', name='r2unet')

  #compile the model
  model.compile(loss='binary_crossentropy', optimizer=Adam(lr = 1e-3), 
              metrics=['accuracy', IoU])

  #generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  #fit model on data
  resunet_history = model.fit(image_dataset[train], mask_dataset[train], 
                    verbose=1,
                    batch_size = batch_size,
                    validation_data=(image_dataset[test], mask_dataset[test]), 
                    shuffle=False,
                    epochs=epochs,
                    callbacks=callbacks)

  #append evaluation values for every fold to a list
  acc_per_fold.append(model.evaluate(image_dataset[test], mask_dataset[test])[1])
  loss_per_fold.append(model.evaluate(image_dataset[test], mask_dataset[test])[0])
  IoU_per_fold.append(model.evaluate(image_dataset[test], mask_dataset[test])[2])

  #increase fold number
  fold_no += 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]} - IoU: {IoU_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print(f'> IoU: {np.mean(IoU_per_fold)}')
print('------------------------------------------------------------------------')

FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: 'D:/UniBas/Bachelorarbeit/DeepACSA_images_VL/little_images/'